In [7]:
codon_chart = {'AUG': 'M', 'UAA': 'STOP', 'UAG': 'STOP', 'UGA': 'STOP', 
               'UUU': 'F', 'UUC': 'F', 'UUA': 'L', 'UUG': 'L',
               'UCU': 'S', 'UCC': 'S', 'UCA': 'S', 'UCG': 'S',
              'UAU': 'Y', 'UAC': 'Y', 'UGU': 'C', 'UGC': 'C', 'UGG': 'W',
              'CUU': 'L', 'CUC': 'L', 'CUA': 'L', 'CUG': 'L',
              'CCU': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
              'CAU': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
              'CGU': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
              'AUU': 'I', 'AUC': 'I', 'AUA': 'I',
              'ACU': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
              'AAU': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
              'AGU': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
              'GUU': 'V', 'GUC': 'V', 'GUA': 'V', 'GUG': 'V',
              'GCU': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
              'GAU': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
              'GGU': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'}

dna_seq = input('Please type your DNA sequence:')
def dnaseq_aminoacidseq(dna_seq):
    rna_seq = dna_seq.replace('T', 'U') # Convert to mRNA seq
    amino_acids = []
    for i in range(0, len(rna_seq), 3): # Range from 1st to last index, in steps of 3 (bc every 3 nucleotides is 1 codon)
        codon = rna_seq[i:i+3]
        if codon in codon_chart:
            amino_acid = codon_chart[codon]
            if amino_acid == 'STOP':
                break
            amino_acids.append(amino_acid)
    amino_acids_string = ''.join([str(element) for element in amino_acids]) # Converting to string
    return amino_acids_string

amino_acid_seq = dnaseq_aminoacidseq(dna_seq)
print(f"Amino Acid Sequence: {amino_acid_seq}")

# Can use .transcribe() and .translate()

Please type your DNA sequence: ATGGTGACCTCTGAGGAGCCTGTTGCCACGTCACGTTGACACCTGAGGACGGTGGTGCC


Amino Acid Sequence: MVTSEEPVATSR


In [8]:
from Bio import Blast # Blast module from Biopython

result_stream = Blast.qblast('blastp', 'nr', 'MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPK')
# qblast passes sequence data as the main parameter
# Internal programme: blastp for protein. Otherwise, blastn is for nucleotides
# Database: nr stands for non-redundant. All identical sequences are represented by a single entry in the database. Allows to access all identical sequences in the database
# Results are given in XML format by default

with open("my_blast.xml", "wb") as out_stream:
    out_stream.write(result_stream.read())
# Results are read and written to a file
# wb: File is opened in write-binary mode

result_stream = open("my_blast.xml", "rb")
# rb: File is opened in read-binary mode

blast_record = Blast.read(result_stream)
# File is opened and read in BLAST

print(blast_record[0], # Access first record
      '\n\n', blast_record[0].target.name, # Sequence name/ identifier
      '\n\n', blast_record[0][0]) # Access the first record (though only one query is input in this example) and first hit (alignment)
# There are multiple hits in 1 record

Query: Query_3508321
       unnamed protein product
  Hit: gb|KAI2558339.1| (length=90)
       hemoglobin subunit beta, partial [Homo sapiens] >gb|KAI4069683.1|
       hemoglobin subunit beta, partial [Homo sapiens] >gb|UXQ07652.1|
       hemoglobin subunit beta, partial [Homo sapiens]
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0   5.5e-36     126.72      60           [0:60]                 [0:60] 

 KAI2558339 

 Query : Query_3508321 Length: 60 Strand: Plus
        unnamed protein product
Target: gb|KAI2558339.1| Length: 90 Strand: Plus
        hemoglobin subunit beta, partial [Homo sapiens] >gb|KAI4069683.1|
        hemoglobin subunit beta, partial [Homo sapiens] >gb|UXQ07652.1|
        hemoglobin subunit beta, partial [Homo sapiens]

Score:126 bits(317), Expect:5e-36,
Identit

In [20]:
import requests
import nglview as nv

pdb_ids = [blast_record[0].target.name.split('_')[0]] # Create PDB ID list

if pdb_ids:  # If list is not empty
    first_pdb_id = pdb_ids[0]  # First result
    print(f"PDB ID: {first_pdb_id}")  # Print the PDB ID

    # Construct the URL to download the PDB file
    pdb_url = f"https://files.rcsb.org/download/{first_pdb_id}.pdb"
    response = requests.get(pdb_url)  # Request to download the PDB file

    if response.status_code == 200:  # Check for successful request
        with open(f"{first_pdb_id}.pdb", "w") as pdb_file:  # Open file in write mode
            pdb_file.write(response.text)  # Write the content to the file

        # Visualise using nglview
        view = nv.show_file(f"{first_pdb_id}.pdb")
        view.add_representation('cartoon')
        view

PDB ID: KAI2558339


In [24]:
view = nv.show_pdbid("1A3N")
view.add_representation('cartoon')
view

NGLWidget()